In [2]:
import cv2
import numpy as np
import pytesseract
import os

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\tesseract-ocr\tesseract.exe'

In [ ]:
def annotate_handwritten_text(image):
    # Преобразование изображения в оттенки серого
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Шумоподавление
    denoised = cv2.fastNlMeansDenoising(gray, None, 10, 7, 15)

    # Пороговая бинаризация
    thresh = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 31, 25)

    # Нахождение контуров
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

    # Список для сохранения изображений с текстом
    images_with_text = []

    # Конфигурации для Tesseract
    custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ0123456789 -l rus --dpi 300'

    for contour in contours:
        # Ограничивающий прямоугольник
        x, y, w, h = cv2.boundingRect(contour)

        # Отсекаем область интереса
        roi = image[y:y + h, x:x + w]

        # Распознавание текста
        text = pytesseract.image_to_string(roi, config=custom_config)

        # Если текст найден, добавляем область в список
        if text.strip():
            images_with_text.append(roi)

    # Возвращаем список найденных изображений с текстом
    return images_with_text

In [ ]:
folder_path = "photos"
output_folder = "tagged_photos"
image_list = []

In [ ]:
#добавляем фото и название в список
for filename in os.listdir(folder_path):
    img = cv2.imread(os.path.join(folder_path, filename))
    image_list.append((filename, img))

In [ ]:
#Сохраняем новые фотографии
for filename, img in image_list:
    box_images = annotate_handwritten_text(img)
    k = 0
    for i in box_images:
        k +=1
        output_path = os.path.join(output_folder, filename + str(k))
        cv2.imwrite(output_path, i)